# En proceso por favor no tomar nada de este file

In [ ]:
def create_message(message, conversation_history=None):
    if conversation_history is None:
        conversation_history = []

    messages = [
        {
            "role": "system",
            "content": (
                "You are a friendly assistant trained to understand the user's mood "
                "through casual and supportive conversation. "
                "Ask follow-up questions to gently confirm how the user is feeling."
            )
        },
        *conversation_history,
        {"role": "user", "content": message}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content

def mood_analysis(message, conversation_history=None):
    if conversation_history is None:
        conversation_history = []

    messages = [
        {
            "role": "system",
            "content": (
                "Analyze the user's message and respond with only one word "
                "indicating the detected mood: Party, Melancholic, Joyful, Sad, Motivating, Relaxed, or Undetected."
            )
        },
        *conversation_history,
        {"role": "user", "content": message}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content

def create_message_music(detected_mood, message, conversation_history=None):
    if conversation_history is None:
        conversation_history = []

    messages = [
        {
            "role": "system",
            "content": (
                f"You are a friendly and conversational assistant that helps users choose the kind of music they want to listen to. "
                f"The user is currently feeling '{detected_mood}'. Based on that and the conversation so far, "
                "have a natural and empathetic dialogue to discover what kind of music mood they would prefer right now. "
                "You are not classifying the music mood yourself — your goal is to guide the conversation and understand the user's preference. "
                "The available music moods are: Party, Melancholic, Joyful, Sad, Motivating or Relaxed"
                "Make the interaction feel human, supportive, and casual."
            )
        },
        *conversation_history,
        {"role": "user", "content": message}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content

def recommend_music_mood(detected_mood, message, conversation_history=None):
    if conversation_history is None:
        conversation_history = []

    messages = [
        {
            "role": "system",
            "content": (
                f"The user is currently feeling '{detected_mood}'. "
                "Based on this and the conversation history, determine what type of music mood they would prefer. "
                "Respond with only one with one word of the following: Party, Melancholic, Joyful, Sad, Motivating, Relaxed, or Undetected."
            )
        },
        *conversation_history,
        {"role": "user", "content": message}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content.strip()

def recommend_tack(detected_mood, detected_mood_music, message, conversation_history=None):
    if conversation_history is None:
        conversation_history = []

    messages = [
        {
            "role": "system",
            "content": (
                "You are a kind and empathetic assistant. "
                f"The user is currently feeling '{detected_mood}', but they prefer to listen to '{detected_mood_music}' music right now. "
                "Based on this and the conversation history, write a warm, natural, and supportive message. "
                f"You Must recommend this song {song_name} - {song_artist} with the Spotify track link ({spotify_link}) as a recommendation that matches their preferred music mood. "
                "Make it feel personal and encouraging, not robotic."
            )
        },
        *conversation_history,
        {"role": "user", "content": message}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content.strip()